In [1]:
import pandas as pd
from pathlib import Path

In [2]:
path1=Path("/Users/zainabnazari/data_work")

In [3]:
file_no = pd.read_csv(path1/"only_6M_12M_mRNA_DEGs_FDR_5pc_1_5fc_filtered_Log2CPM_TMM_Norm_HtSeq_Union_paired_reads_counts_GTF_gene.txt",delimiter='\t')

In [8]:
file_no['SampleID']

0      ENSMUSG00000020660
1      ENSMUSG00000059305
2      ENSMUSG00000045394
3      ENSMUSG00000115625
4      ENSMUSG00000045337
              ...        
358    ENSMUSG00000003379
359    ENSMUSG00000040592
360    ENSMUSG00000121471
361    ENSMUSG00000102610
362    ENSMUSG00000142531
Name: SampleID, Length: 363, dtype: object

In [9]:
file_no['SampleID'].to_csv(path1/'list.txt', header=False,index=False)

In [13]:
file_61 = pd.read_csv(path1/'only_6M_12M_miRNA_DEGs_FDR_5pc_1_5fc_Bowtie1_align_miRBase_v_0_m_1_only_R1_samtools_idxstat_filtered_Log2CPM_TMM_Norm.txt', delimiter='\t')

In [12]:
!pip install python-igraph



INFO: pip is looking at multiple versions of python-igraph to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 3.1 MB/s eta 0:00:00a 0:00:01


In [15]:
import requests
import pandas as pd
import igraph as ig
import matplotlib.pyplot as plt

# Replace with your actual list of Ensembl Gene IDs
ensembl_ids = list(file_no['SampleID'])  # Example genes

# Define STRING API parameters
STRING_API_URL = "https://string-db.org/"
OUTPUT_FORMAT = "tsv"
METHOD = "network"

In [24]:
# Organism (9606 = Human, change for other species)
species = 10090

# Convert list to a comma-separated string
genes_str = ",".join(map(str, ensembl_ids))

In [25]:

# Build API request URL
request_url = f"{STRING_API_URL}/{OUTPUT_FORMAT}/{METHOD}?identifiers={genes_str}&species={species}&network_type=functional"


In [26]:

# Fetch data from STRING
response = requests.get(request_url)

In [29]:





# Check if request was successful
if response.status_code == 200:
    # Read the STRING interactions into a DataFrame
    data = pd.read_csv(pd.compat.StringIO(response.text), sep="\t")

    # Extract relevant columns (protein1, protein2, and combined_score)
    interactions = data[['TP53', 'BRCA1', '982']]
    
    # Convert combined score to weights (scale between 0 and 1)
    interactions['weight'] = interactions['combined_score'] / 1000

    # Create an igraph graph from the DataFrame
    g = ig.Graph.TupleList(interactions.itertuples(index=False), weights=True, directed=False)

    # Plot the network
    ig.plot(g, layout=g.layout("fr"), vertex_label=g.vs["name"], edge_width=[e["weight"] * 5 for e in g.es])
    
else:
    print(f"Error fetching data: {response.status_code}")


Error fetching data: 404


In [30]:
import igraph as ig

# Step 1: Prepare your data
# Example list of ensemble gene IDs
gene_ids = ['ENSG000001', 'ENSG000002', 'ENSG000003', 'ENSG000004']

# Example edges between genes (these should represent some relationship)
edges = [('ENSG000001', 'ENSG000002'),
         ('ENSG000001', 'ENSG000003'),
         ('ENSG000002', 'ENSG000004')]

# Step 2: Create a graph
# Create a graph object
g = ig.Graph()

# Add vertices (gene IDs)
g.add_vertices(gene_ids)

# Add edges
# Ensure the gene IDs in edges are part of the vertices
g.add_edges(edges)

# Step 3: Plot the graph
ig.plot(g, vertex_label=g.vs['name'], vertex_size=20, edge_arrow_size=0.5)


AttributeError: Plotting not available; please install pycairo or cairocffi

In [14]:
file_61

,SampleID,6M_CD_S315,6M_HL_S316,6M_MN_S317,6M_OP_S318,6M_QR_S319,6M_ST_S320,12M_AB_S321,12M_CD_S322,12M_EF_S323,12M_GH_S324,12M_IJ_S325,12M_KL_S326
0,mmu-miR-1264-3p,9.687629,11.883774,11.778448,11.194754,11.311906,11.122253,9.138035,8.830678,8.681971,9.497815,8.823775,8.394148
1,mmu-miR-1298-5p,10.490571,12.217356,11.942825,11.489104,11.331554,11.173579,9.589777,9.583212,9.120731,10.518709,9.362863,8.877956
2,mmu-miR-5099,2.648121,4.850137,3.865312,5.400656,4.509685,4.998824,3.456963,2.743877,2.856106,2.509578,2.484597,2.145687
3,mmu-miR-448-3p,7.938014,9.217713,8.906007,8.180803,8.663388,8.308246,6.858872,6.861356,6.616046,7.587441,6.595774,6.597387
4,mmu-miR-20a-3p,2.199721,4.975961,5.387412,5.502184,5.291638,2.756381,4.325130,3.349624,2.507817,3.028898,2.252924,2.947137
...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,mmu-miR-874-3p,8.096462,6.457200,6.872207,7.407734,6.298491,7.322416,8.124092,8.275673,8.277916,8.296745,8.191097,8.137713
57,mmu-miR-491-5p,4.826458,2.966234,4.677634,4.457272,2.616025,4.080345,4.926115,5.292914,5.024781,5.021597,5.262794,5.646157
58,mmu-miR-92b-5p,5.833114,4.658695,4.971026,2.875250,3.434719,4.517470,5.561759,5.802337,5.775109,5.682013,5.936092,6.094695
59,mmu-miR-770-5p,7.362103,4.631137,6.575542,6.715974,4.258353,5.510334,7.100339,7.340368,7.490075,7.387850,7.435542,7.844091


In [17]:
from IPython.display import display

In [28]:
file_result = pd.read_csv("miRNA_targets_results.csv",delimiter=',')


In [30]:
pd.set_option('display.max_rows', None)

In [31]:
pd.set_option('display.max_columns', None)

In [4]:
import sys
print(sys.executable)

/Users/zainabnazari/opt/miniconda3/bin/python


In [5]:
import sys
print(sys.executable)
import rpy2
print(rpy2.__version__)

/Users/zainabnazari/opt/miniconda3/bin/python
3.5.17


In [2]:
%pip install rpy2

Note: you may need to restart the kernel to use updated packages.


In [6]:
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
pandas2ri.activate() #activate pandas conversion.
r_df = robjects.r['your_r_dataframe'] #get your r dataframe
py_df = pandas2ri.ri2py(r_df) #convert r dataframe to pandas dataframe.
print(type(py_df))

# Initialize R
robjects.r['options'](warn=-1)  # Suppress warnings

# Install and load the multiMiR package
utils = importr('utils')
utils.chooseCRANmirror(r=robjects.r)
utils.install_packages('multiMiR')
multiMiR = importr('multiMiR')

# Define miRNA and databases
mirna = "mmu-miR-187-3p"
predicted_dbs = robjects.StrVector([
    "diana_microt", "elmmo", "microcosm", "miranda", "mirdb", "pictar", "pita", "targetscan"
])

# Function to get targets from a database
def get_targets_from_db(mirna, db):
    result = multiMiR.get_multimir(org="mmu", mirna=mirna, table=db)
    if not robjects.r.is_null(result):
        data = robjects.r.slot(result, "data")
        if not robjects.r.is_null(data) and robjects.r("nrow")(data)[0] > 0:
            return robjects.r.slot(data, "mRNA_Gene")
    return robjects.StrVector([])

# Get targets from all predicted databases
all_targets = set()
for db in predicted_dbs:
    targets = get_targets_from_db(mirna, db)
    all_targets.update(list(targets))

# Print results
print(f"Number of predicted targets for {mirna}: {len(all_targets)}")
#if len(all_targets) < 50: # if there are not too many genes, print them.
#  print(f"Predicted targets: {all_targets}")

NotImplementedError: Conversion 'py2rpy' not defined for objects of type '<class 'rpy2.robjects.R'>'

In [7]:
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
pandas2ri.activate() #activate pandas conversion.
r_df = robjects.r['your_r_dataframe'] #get your r dataframe
py_df = pandas2ri.ri2py(r_df) #convert r dataframe to pandas dataframe.
print(type(py_df))

KeyError: "'your_r_dataframe' not found"